In [1]:
# coding=utf-8
"""
This module contains the tests for the exchanges classes
"""
from fastlane_bot import Bot, Config
from fastlane_bot.bot import CarbonBot
from fastlane_bot.tools.cpc import ConstantProductCurve
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC
from fastlane_bot.events.exchanges import UniswapV2, UniswapV3, SushiswapV2, CarbonV1, BancorV3
from fastlane_bot.events.interface import QueryInterface
from fastlane_bot.helpers.poolandtokens import PoolAndTokens
from fastlane_bot.helpers import TradeInstruction, TxReceiptHandler, TxRouteHandler, TxSubmitHandler, TxHelpers, TxHelper
from fastlane_bot.events.managers.manager import Manager
from fastlane_bot.events.interface import QueryInterface
import decimal
import random
import time
from _decimal import Decimal
from dataclasses import dataclass, asdict, field
from datetime import datetime
from typing import List, Dict, Tuple, Any, Callable
from typing import Optional

from fastlane_bot.config import Config
from fastlane_bot.helpers import (
    TxSubmitHandler,
    TxSubmitHandlerBase,
    TxReceiptHandler,
    TxReceiptHandlerBase,
    TxRouteHandler,
    TxRouteHandlerBase,
    TxHelpers,
    TxHelpersBase,
    TradeInstruction,
    Univ3Calculator,
    RouteStruct,
)
from fastlane_bot.helpers.routehandler import maximize_last_trade_per_tkn #,BalancerInputTooLargeError
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC, CPCContainer, T
from fastlane_bot.tools.optimizer import CPCArbOptimizer
from fastlane_bot.events.interface import QueryInterface
from fastlane_bot.modes.pairwise_multi import FindArbitrageMultiPairwise
from fastlane_bot.modes.pairwise_single import FindArbitrageSinglePairwise
from fastlane_bot.modes.triangle_multi import ArbitrageFinderTriangleMulti
from fastlane_bot.modes.triangle_single import ArbitrageFinderTriangleSingle
from fastlane_bot.modes.triangle_single_bancor3 import ArbitrageFinderTriangleSingleBancor3
from fastlane_bot.modes.triangle_bancor_v3_two_hop import ArbitrageFinderTriangleBancor3TwoHop
from fastlane_bot.utils import num_format, log_format, num_format_float
from joblib import Parallel, delayed
from typing import List, Dict
import pytest
import math
import json
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV3))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(SushiswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonV1))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(BancorV3))
from fastlane_bot.testing import *
from fastlane_bot.modes import triangle_single_bancor3
#plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)

ConstantProductCurve v3.3.1 (05/Oct/2023)
CarbonBot v3-b2.2 (20/June/2023)
UniswapV2 v0.0.2 (2023-08-27)
UniswapV3 v0.0.2 (2023-08-27)
SushiswapV2 v0.0.2 (2023-08-27)
CarbonV1 v0.0.2 (2023-08-27)
BancorV3 v0.0.2 (2023-08-27)
imported m, np, pd, plt, os, sys, decimal; defined iseq, raises, require
Version = 3-b2.2 [requirements >= 3.0 is met]


In [2]:
C = cfg = Config.new(config=Config.CONFIG_MAINNET,) #blockchain="coinbase_base")
C.DEFAULT_MIN_PROFIT_GAS_TOKEN = 0.0001

setup_bot = CarbonBot(ConfigObj=C)
pools = None

### 
"""
Put path to log file here >>>
"""
###

path = os.path.normpath("fastlane_bot/data/tests/latest_pool_data_testing_balancer.json")
print(f"path={path}")
with open(path) as f:
    pools = json.load(f)



# with open("fastlane_bot/data/tests/latest_pool_data_testing_balancer.json") as f:
#     pools = json.load(f)

flashloan_tokens = ["BNT-FF1C","WETH-6Cc2","USDC-eB48","LINK-86CA"]
#flashloan_tokens = ["WETH-0006,USDC-2913"]

pools = [pool for pool in pools]
pools[0]
static_pools = pools
state = pools
exchanges = list({ex['exchange_name'] for ex in state})
db = QueryInterface(state=state, ConfigObj=C, exchanges=exchanges)
setup_bot.db = db

static_pool_data_filename = "static_pool_data"

static_pool_data = pd.read_csv(f"fastlane_bot/data/{static_pool_data_filename}.csv", low_memory=False)
    
uniswap_v2_event_mappings = pd.read_csv("fastlane_bot/data/uniswap_v2_event_mappings.csv", low_memory=False)
uniswap_v3_event_mappings = pd.read_csv("fastlane_bot/data/uniswap_v3_event_mappings.csv", low_memory=False)

tokens = pd.read_csv("fastlane_bot/data/tokens.csv", low_memory=False)
        
exchanges = "carbon_v1,pancakeswap_v2,pancakeswap_v3,uniswap_v2,uniswap_v3"

exchanges = exchanges.split(",")


alchemy_max_block_fetch = 20
static_pool_data["cid"] = [
        cfg.w3.keccak(text=f"{row['descr']}").hex()
        for index, row in static_pool_data.iterrows()
    ]
# Filter out pools that are not in the supported exchanges
static_pool_data = [
    row for index, row in static_pool_data.iterrows()
    if row["exchange_name"] in exchanges
]

static_pool_data = pd.DataFrame(static_pool_data)
static_pool_data['exchange_name'].unique()
# Initialize data fetch manager
mgr = Manager(
    web3=cfg.w3,
    cfg=cfg,
    pool_data=static_pool_data.to_dict(orient="records"),
    SUPPORTED_EXCHANGES=exchanges,
    alchemy_max_block_fetch=alchemy_max_block_fetch,
    uniswap_v2_event_mappings=uniswap_v2_event_mappings,
    #uniswap_v3_event_mappings=uniswap_v3_event_mappings,
    tokens=tokens.to_dict(orient="records"),
)

# Add initial pools for each row in the static_pool_data
start_time = time.time()
Parallel(n_jobs=-1, backend="threading")(
    delayed(mgr.add_pool_to_exchange)(row) for row in mgr.pool_data
)
cfg.logger.info(f"Time taken to add initial pools: {time.time() - start_time}")

# check if any duplicate cid's exist in the pool data
mgr.deduplicate_pool_data()
cids = [pool["cid"] for pool in mgr.pool_data]
assert len(cids) == len(set(cids)), "duplicate cid's exist in the pool data"
def init_bot(mgr: Manager) -> CarbonBot:
    """
    Initializes the bot.

    Parameters
    ----------
    mgr : Manager
        The manager object.

    Returns
    -------
    CarbonBot
        The bot object.
    """
    mgr.cfg.logger.info("Initializing the bot...")
    bot = CarbonBot(ConfigObj=mgr.cfg)
    bot.db = db
    bot.db.mgr = mgr
    assert isinstance(
        bot.db, QueryInterface
    ), "QueryInterface not initialized correctly"
    return bot
bot = init_bot(mgr)
# add data cleanup steps from main.py
bot.db.handle_token_key_cleanup()
#bot.db.remove_unmapped_uniswap_v2_pools()
bot.db.remove_zero_liquidity_pools()
bot.db.remove_unsupported_exchanges()
tokens = bot.db.get_tokens()
ADDRDEC = {t.key: (t.address, int(t.decimals)) for t in tokens if not math.isnan(t.decimals)}
#flashloan_tokens = bot.setup_flashloan_tokens(None)
#flashloan_tokens = ['WBTC-2c599', 'USDC-eB48', 'LINK-86CA', 'USDT-1ec7']


CCm = bot.setup_CCm(None)
pools = db.get_pool_data_with_tokens()


2023-11-21 11:24:46,954 [fastlane:INFO] - 
2023-11-21 11:24:46,955 [fastlane:INFO] - **********************************************
2023-11-21 11:24:46,955 [fastlane:INFO] - The logging path is set to: logs/20231121-112446\bot.log
2023-11-21 11:24:46,955 [fastlane:INFO] - **********************************************
2023-11-21 11:24:46,956 [fastlane:INFO] - 


path=fastlane_bot\data\tests\latest_pool_data_testing_balancer.json


2023-11-21 11:24:49,713 [fastlane:INFO] - Retrieved 208 carbon pairs from contract
2023-11-21 11:24:51,281 [fastlane:INFO] - Time taken to add initial pools: 0.10045790672302246
2023-11-21 11:24:51,286 [fastlane:INFO] - Initializing the bot...
2023-11-21 11:24:51,492 [fastlane:INFO] - uniswap_v3: 470
2023-11-21 11:24:51,493 [fastlane:INFO] - sushiswap_v2: 63
2023-11-21 11:24:51,494 [fastlane:INFO] - uniswap_v2: 708
2023-11-21 11:24:51,494 [fastlane:INFO] - bancor_v2: 0
2023-11-21 11:24:51,495 [fastlane:INFO] - bancor_v3: 27
2023-11-21 11:24:51,495 [fastlane:INFO] - bancor_pol: 51
2023-11-21 11:24:51,497 [fastlane:INFO] - carbon_v1: 185
2023-11-21 11:24:51,497 [fastlane:INFO] - pancakeswap_v2: 0
2023-11-21 11:24:51,497 [fastlane:INFO] - pancakeswap_v3: 0
2023-11-21 11:24:51,497 [fastlane:INFO] - balancer: 63
2023-11-21 11:24:51,628 [fastlane:INFO] - uniswap_v3_zero_liquidity_pools: 1024
2023-11-21 11:24:51,629 [fastlane:INFO] - sushiswap_v2_zero_liquidity_pools: 39
2023-11-21 11:24:51,6

# Test Native Gas Token Wrap Unwrap [NB061]

## Test_Wrap_Unwrap_Gas_Token_In_Route_Struct

In [3]:
arb_mode = "multi"

arb_finder = bot._get_arb_finder(arb_mode)
finder = arb_finder(
            flashloan_tokens=flashloan_tokens,
            CCm=CCm,
            mode="bothin",
            result=bot.AO_CANDIDATES,
            ConfigObj=bot.ConfigObj,
        )
r = finder.find_arbitrage()

r = [arb for arb in r if len(arb[2]) >= 2]
r.sort(key=lambda x: x[0], reverse=True)



number of arbs found = 18


In [11]:
for arb in r:

    (
        best_profit,
        best_trade_instructions_df,
        best_trade_instructions_dic,
        best_src_token,
        best_trade_instructions,
    ) = arb

    # Order the trade instructions
    (
        ordered_trade_instructions_dct,
        tx_in_count,
    ) = bot._simple_ordering_by_src_token(
        best_trade_instructions_dic, best_src_token
    )

    # Scale the trade instructions
    ordered_scaled_dcts = bot._basic_scaling(
        ordered_trade_instructions_dct, best_src_token
    )

    # Convert the trade instructions
    ordered_trade_instructions_objects = bot._convert_trade_instructions(
        ordered_scaled_dcts
    )

    # Create the tx route handler
    tx_route_handler = bot.TxRouteHandlerClass(
        trade_instructions=ordered_trade_instructions_objects
    )

    # Aggregate the carbon trades
    agg_trade_instructions = (
        tx_route_handler.aggregate_carbon_trades(ordered_trade_instructions_objects)
        if bot._carbon_in_trade_route(ordered_trade_instructions_objects)
        else ordered_trade_instructions_objects
    )

    # Calculate the trade instructions
    #try:
    calculated_trade_instructions = tx_route_handler.calculate_trade_outputs(trade_instructions=agg_trade_instructions)

    # Aggregate multiple Bancor V3 trades into a single trade
    calculated_trade_instructions = tx_route_handler.aggregate_bancor_v3_trades(
        calculated_trade_instructions
    )
    flashloan_struct = tx_route_handler.generate_flashloan_struct(
        trade_instructions_objects=calculated_trade_instructions)

    # Get the flashloan token
    fl_token = calculated_trade_instructions[0].tknin_key


    best_profit = flashloan_tkn_profit = tx_route_handler.calculate_trade_profit(
        calculated_trade_instructions
    )


    # Use helper function to calculate profit
    best_profit, flt_per_bnt, profit_usd = bot.calculate_profit(
        CCm, best_profit, fl_token
    )

    # Log the best trade instructions
    bot.handle_logging_for_trade_instructions(
        1, best_profit=best_profit  # The log id
    )

    # Use helper function to update the log dict
    log_dict = bot.update_log_dict(
        arb_mode,
        best_profit,
        profit_usd,
        flashloan_tkn_profit,
        calculated_trade_instructions,
        fl_token,
    )

    # Log the log dict
    bot.handle_logging_for_trade_instructions(2, log_dict=log_dict)  # The log id

    # Check if the best profit is greater than the minimum profit
    # if best_profit < bot.ConfigObj.DEFAULT_MIN_PROFIT:
    #     bot.ConfigObj.logger.info(
    #         f"Opportunity with profit: {num_format(best_profit)} does not meet minimum profit: {bot.ConfigObj.DEFAULT_MIN_PROFIT}, discarding."
    #     )


    # Get the flashloan amount and token address
    flashloan_amount = int(calculated_trade_instructions[0].amtin_wei)
    flashloan_token_address = bot.ConfigObj.w3.toChecksumAddress(
        bot.db.get_token(key=fl_token).address
    )

    # Log the flashloan amount and token address
    bot.handle_logging_for_trade_instructions(
        3,  # The log id
        flashloan_amount=flashloan_amount,
    )

    # Encode the trade instructions
    encoded_trade_instructions = tx_route_handler.custom_data_encoder(
        calculated_trade_instructions
    )

    # Get the deadline
    deadline = bot._get_deadline(None)

    # Get the route struct
    route_struct = [
        asdict(rs)
        for rs in tx_route_handler.get_route_structs(
            encoded_trade_instructions, deadline
        )
    ]
    route_struct = maximize_last_trade_per_tkn(route_struct=route_struct)
    # Check if the result is None

    route_struct = tx_route_handler.add_wrap_or_unwrap_trades_to_route(trade_instructions=calculated_trade_instructions, route_struct=route_struct, flashloan_struct=flashloan_struct)
    
    #print(f"\nNEW ROUTE STRUCT:\nflashloan_struct={flashloan_struct}\nroute_struct={route_struct}\n\n")
    assert flashloan_struct[0]["sourceTokens"][0] in route_struct[0]["sourceToken"]
    assert flashloan_struct[0]["sourceTokens"][0] in route_struct[-1]["targetToken"]

    for idx, route in enumerate(route_struct):
        if idx == 0:
            continue
        assert route["sourceToken"] == route_struct[idx - 1]["targetToken"]


    # Get the cids
    cids = list({ti["cid"].split("-")[0] for ti in best_trade_instructions_dic})


    # Log the route_struct
    bot.handle_logging_for_trade_instructions(
        4,  # The log id
        flashloan_amount=flashloan_amount,
        flashloan_token_address=flashloan_token_address,
        route_struct=route_struct,
        best_trade_instructions_dic=best_trade_instructions_dic,
    )


2023-11-21 12:03:48,546 [fastlane:INFO] - [2023-11-21T12:03:48::1700561028] |calculated_arb| == {'type': 'multi', 'profit_gas_token': -11.7576, 'profit_usd': -11.646, 'flashloan': [{'token': 'USDC-eB48', 'amount': 45029.1708, 'profit': -11.7576}], 'trades': [{'trade_index': 0, 'exchange': 'carbon_v1', 'tkn_in': 'USDC-eB48', 'amount_in': 45029.1708, 'tkn_out': 'USDT-1ec7', 'amt_out': 45021.2613, 'cid0': '5304634371'}, {'trade_index': 1, 'exchange': 'uniswap_v3', 'tkn_in': 'USDT-1ec7', 'amount_in': 45021.2613, 'tkn_out': 'USDC-eB48', 'amt_out': 45017.4132, 'cid0': 'ddf5be231a'}]}
2023-11-21 12:03:49,643 [fastlane:INFO] - [2023-11-21T12:03:49::1700561029] |calculated_arb| == {'type': 'multi', 'profit_gas_token': 0.002, 'profit_usd': 3.2568, 'flashloan': [{'token': 'WETH-6Cc2', 'amount': 0.1258, 'profit': 0.002}], 'trades': [{'trade_index': 0, 'exchange': 'uniswap_v2', 'tkn_in': 'WETH-6Cc2', 'amount_in': 0.1258, 'tkn_out': 'BNT-FF1C', 'amt_out': 518.1364, 'cid0': 'acb0f128fe'}, {'trade_ind


NEW ROUTE STRUCT:
flashloan_struct=[{'platformId': 7, 'sourceTokens': ['0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'], 'sourceAmounts': [45029170793]}]
route_struct=[{'platformId': 6, 'sourceToken': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48', 'targetToken': '0xdAC17F958D2ee523a2206206994597C13D831ec7', 'sourceAmount': 0, 'minTargetAmount': 45021261344, 'deadline': 1701211027, 'customAddress': '0xC537e898CD774e2dCBa3B14Ea6f34C93d5eA45e1', 'customInt': 10, 'customData': '0x0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000400000000000000000000000000000003000000000000000000000000000000030000000000000000000000000000000000000000000000000000000005f86a8b00000000000000000000000000000003000000000000000000000000000000b500000000000000000000000000000000000000000000000000000000758f0c7900000000000000000000000000000003000000000000000000000000000001c7000000000000000000000000000000000000000000000000000000000006ad830000000000

2023-11-21 12:03:50,217 [fastlane:INFO] - [2023-11-21T12:03:50::1700561030] |calculated_arb| == {'type': 'multi', 'profit_gas_token': 8.098, 'profit_usd': 3.2177, 'flashloan': [{'token': 'BNT-FF1C', 'amount': 517.2487, 'profit': 8.098}], 'trades': [{'trade_index': 0, 'exchange': 'carbon_v1', 'tkn_in': 'BNT-FF1C', 'amount_in': 517.2487, 'tkn_out': 'WETH-6Cc2', 'amt_out': 0.1275, 'cid0': '9450326076'}, {'trade_index': 1, 'exchange': 'uniswap_v2', 'tkn_in': 'WETH-6Cc2', 'amount_in': 0.1275, 'tkn_out': 'BNT-FF1C', 'amt_out': 525.3467, 'cid0': 'acb0f128fe'}]}



NEW ROUTE STRUCT:
flashloan_struct=[{'platformId': 7, 'sourceTokens': ['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'], 'sourceAmounts': [125758000000000000]}]
route_struct=[{'platformId': 3, 'sourceToken': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', 'targetToken': '0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C', 'sourceAmount': 0, 'minTargetAmount': 518136407003744449948, 'deadline': 1701211027, 'customAddress': '0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D', 'customInt': 3000, 'customData': '0x'}, {'platformId': 6, 'sourceToken': '0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C', 'targetToken': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE', 'sourceAmount': 0, 'minTargetAmount': 127731826500234221, 'deadline': 1701211027, 'customAddress': '0xC537e898CD774e2dCBa3B14Ea6f34C93d5eA45e1', 'customInt': 2000, 'customData': '0x000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000b000000000000000000

2023-11-21 12:03:50,849 [fastlane:INFO] - [2023-11-21T12:03:50::1700561030] |calculated_arb| == {'type': 'multi', 'profit_gas_token': 0.0018, 'profit_usd': 2.9356, 'flashloan': [{'token': 'WETH-6Cc2', 'amount': 0.127, 'profit': 0.0018}], 'trades': [{'trade_index': 0, 'exchange': 'bancor_v3', 'tkn_in': 'WETH-6Cc2', 'amount_in': 0.127, 'tkn_out': 'BNT-FF1C', 'amt_out': 522.5566, 'cid0': '03a3c742d2'}, {'trade_index': 1, 'exchange': 'carbon_v1', 'tkn_in': 'BNT-FF1C', 'amount_in': 522.5566, 'tkn_out': 'WETH-6Cc2', 'amt_out': 0.1288, 'cid0': '9450326076'}]}


...


NEW ROUTE STRUCT:
flashloan_struct=[{'platformId': 7, 'sourceTokens': ['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'], 'sourceAmounts': [174000000000000]}]
route_struct=[{'platformId': 3, 'sourceToken': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', 'targetToken': '0x2b591e99afE9f32eAA6214f7B7629768c40Eeb39', 'sourceAmount': 0, 'minTargetAmount': 8637771351, 'deadline': 1701211027, 'customAddress': '0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D', 'customInt': 3000, 'customData': '0x'}, {'platformId': 6, 'sourceToken': '0x2b591e99afE9f32eAA6214f7B7629768c40Eeb39', 'targetToken': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE', 'sourceAmount': 0, 'minTargetAmount': 450911275010526, 'deadline': 1701211027, 'customAddress': '0xC537e898CD774e2dCBa3B14Ea6f34C93d5eA45e1', 'customInt': 2000, 'customData': '0x000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000002a0000000000000000000000000000005a000

2023-11-21 12:03:59,181 [fastlane:INFO] - [2023-11-21T12:03:59::1700561039] |calculated_arb| == {'type': 'multi', 'profit_gas_token': 0.0001, 'profit_usd': 0.1889, 'flashloan': [{'token': 'WETH-6Cc2', 'amount': 0.003, 'profit': 0.0001}], 'trades': [{'trade_index': 0, 'exchange': 'uniswap_v2', 'tkn_in': 'WETH-6Cc2', 'amount_in': 0.003, 'tkn_out': 'SNX-2a6F', 'amt_out': 2.4124, 'cid0': '8394abb00a'}, {'trade_index': 1, 'exchange': 'carbon_v1', 'tkn_in': 'SNX-2a6F', 'amount_in': 2.4124, 'tkn_out': 'WETH-6Cc2', 'amt_out': 0.0031, 'cid0': '8529762546'}]}
2023-11-21 12:03:59,761 [fastlane:INFO] - [2023-11-21T12:03:59::1700561039] |calculated_arb| == {'type': 'multi', 'profit_gas_token': 0.0001, 'profit_usd': 0.1783, 'flashloan': [{'token': 'WETH-6Cc2', 'amount': 0.003, 'profit': 0.0001}], 'trades': [{'trade_index': 0, 'exchange': 'uniswap_v3', 'tkn_in': 'WETH-6Cc2', 'amount_in': 0.003, 'tkn_out': 'SNX-2a6F', 'amt_out': 2.4121, 'cid0': 'bcf6189d22'}, {'trade_index': 1, 'exchange': 'carbon_v1'


NEW ROUTE STRUCT:
flashloan_struct=[{'platformId': 7, 'sourceTokens': ['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'], 'sourceAmounts': [3015000000000000]}]
route_struct=[{'platformId': 3, 'sourceToken': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', 'targetToken': '0xC011a73ee8576Fb46F5E1c5751cA3B9Fe0af2a6F', 'sourceAmount': 0, 'minTargetAmount': 2412354235507889649, 'deadline': 1701211027, 'customAddress': '0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D', 'customInt': 3000, 'customData': '0x'}, {'platformId': 6, 'sourceToken': '0xC011a73ee8576Fb46F5E1c5751cA3B9Fe0af2a6F', 'targetToken': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE', 'sourceAmount': 0, 'minTargetAmount': 3129475406309406, 'deadline': 1701211027, 'customAddress': '0xC537e898CD774e2dCBa3B14Ea6f34C93d5eA45e1', 'customInt': 2000, 'customData': '0x0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000054000000000000000000000000

2023-11-21 12:04:00,544 [fastlane:INFO] - [2023-11-21T12:04:00::1700561040] |calculated_arb| == {'type': 'multi', 'profit_gas_token': 0.0001, 'profit_usd': 0.1151, 'flashloan': [{'token': 'WETH-6Cc2', 'amount': 0.003, 'profit': 0.0001}], 'trades': [{'trade_index': 0, 'exchange': 'balancer', 'tkn_in': 'WETH-6Cc2', 'amount_in': 0.003, 'tkn_out': 'SNX-2a6F', 'amt_out': 2.3948, 'cid0': '95f77b67a8'}, {'trade_index': 1, 'exchange': 'carbon_v1', 'tkn_in': 'SNX-2a6F', 'amount_in': 2.3948, 'tkn_out': 'WETH-6Cc2', 'amt_out': 0.0031, 'cid0': '8529762546'}]}



NEW ROUTE STRUCT:
flashloan_struct=[{'platformId': 7, 'sourceTokens': ['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'], 'sourceAmounts': [3037000000000000]}]
route_struct=[{'platformId': 7, 'sourceToken': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', 'targetToken': '0xC011a73ee8576Fb46F5E1c5751cA3B9Fe0af2a6F', 'sourceAmount': 0, 'minTargetAmount': 2394837388571587690, 'deadline': 1701211027, 'customAddress': '0xC011a73ee8576Fb46F5E1c5751cA3B9Fe0af2a6F', 'customInt': 3249374757253751268355276259122675239274040171040535564758290851987386794023, 'customData': '0x'}, {'platformId': 6, 'sourceToken': '0xC011a73ee8576Fb46F5E1c5751cA3B9Fe0af2a6F', 'targetToken': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE', 'sourceAmount': 0, 'minTargetAmount': 3106751321729970, 'deadline': 1701211027, 'customAddress': '0xC537e898CD774e2dCBa3B14Ea6f34C93d5eA45e1', 'customInt': 2000, 'customData': '0x0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000